In [1]:
# Import necessary libraries
import sys
import zipfile
import os
import logging
from os.path import isdir, join, normpath, split
from pyproj import Transformer
import fiona
from fiona.transform import transform_geom

# Configure logging for consistent message output
logging.basicConfig(level=logging.INFO, format='[py] %(message)s')

# Function to log messages
def logMessage(msg):
    logging.info(msg)

# Print a message to confirm imports and logging configuration
logMessage("Libraries imported and logging configured.")


[py] Libraries imported and logging configured.


In [2]:
# Function to extract the contents of a ZIP file
def unzipfiles(path, zip):
    shapefileName = ''
    if not isdir(path):  # Check if the directory exists
        os.makedirs(path)  # Create directory if it doesn't exist

    for x in zip.namelist():  # Iterate through files in the ZIP
        logMessage(f"Extracting {os.path.basename(x)} ...")
        fileExt = os.path.splitext(x)[1].lower()  # Get file extension
        if fileExt == ".shp":  # Identify shapefiles
            shapefileName = normpath(str(x))

        if not x.endswith('/'):  # Skip directories
            root, name = split(x)
            directory = normpath(join(path, root))
            if not isdir(directory):  # Create necessary directories
                os.makedirs(directory)
            # Write file contents to disk
            with open(join(directory, name), 'wb') as f:
                f.write(zip.read(x))

    return shapefileName  # Return the shapefile name (if found)

# Print a message to confirm the function is ready
logMessage("unzipfiles function defined.")


[py] unzipfiles function defined.


In [3]:
# Wrapper function to handle the ZIP extraction
def unzip(infile, outfol):
    try:
        # Open the ZIP file
        with zipfile.ZipFile(infile, 'r') as zip:
            shapefileName = unzipfiles(outfol, zip)  # Extract files
        return shapefileName
    except Exception as e:
        logMessage(f"Error: {e}")  # Log errors if they occur
        return None

# Print a message to confirm the function is ready
logMessage("unzip function defined.")


[py] unzip function defined.


In [4]:
# Function to reproject a shapefile to a new CRS
def project(input_path, output_path, target_crs):
    try:
        with fiona.open(input_path, 'r') as source:  # Open the source shapefile
            source_crs = source.crs  # Get source CRS
            schema = source.schema  # Get schema (fields, geometry type)

            with fiona.open(
                output_path, 'w',
                driver=source.driver,  # Same driver (e.g., shapefile)
                crs=target_crs,  # New CRS
                schema=schema  # Same schema
            ) as target:
                transformer = Transformer.from_crs(source_crs, target_crs, always_xy=True)
                for feature in source:
                    # Transform geometry to the new CRS
                    transformed_geom = transform_geom(
                        source_crs, target_crs, feature['geometry'], transformer=transformer
                    )
                    feature['geometry'] = transformed_geom
                    target.write(feature)  # Write transformed feature to target
    except Exception as e:
        logMessage(f"Projection error: {e}")  # Log errors
        return None

# Print a message to confirm the function is ready
logMessage("project function defined.")


[py] project function defined.


In [6]:
# Simulated entry point for the script (using test parameters)
if __name__ == '__main__':
    logMessage("Script started")
    try:
        # Example parameters (replace with real ones in production)
        infile = 'archive.zip'  # Input ZIP file
        outdir = './output'  # Output directory
        target_crs = 'EPSG:3857'  # Web Mercator CRS

        if not infile or not outdir:
            logMessage("Usage: script.py <input_zip_file> <output_directory>")
        else:
            # Extract shapefile
            shapefileName = unzip(infile, outdir)
            if shapefileName:
                input_path = join(outdir, shapefileName)
                output_path = join(outdir, f"_{os.path.basename(shapefileName)}")
                # Reproject shapefile
                project(input_path, output_path, target_crs)
                logMessage(f"Shapefile processed and saved to {output_path}")
            else:
                logMessage("No shapefile found.")
    except Exception as e:
        logMessage(f"Unexpected error: {e}")
    logMessage("Finished")


[py] Script started
[py] Extracting TMBPoints.cpg ...
[py] Extracting TMBPoints.dbf ...
[py] Extracting TMBPoints.prj ...
[py] Extracting TMBPoints.sbn ...
[py] Extracting TMBPoints.sbx ...
[py] Extracting TMBPoints.shp ...
[py] Extracting TMBPoints.shp.BRENDANHALLC656.8968.11544.sr.lock ...
[py] Extracting TMBPoints.shp.xml ...
[py] Extracting TMBPoints.shx ...
[py] Projection error: transform_geom() got an unexpected keyword argument 'transformer'
[py] Shapefile processed and saved to ./output/_TMBPoints.shp
[py] Finished
